# Advanced Features

This notebook explores advanced features of Promptomatix including:
- Custom configurations and parameters
- Batch processing and automation
- Integration examples
- Advanced optimization strategies
- Performance monitoring and debugging

## Prerequisites

Make sure you have completed the previous notebooks first.

---

## Setup and Imports

In [ ]:
import os
import sys
import json
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv

# Add the src directory to Python path
sys.path.append('../src')

# Import Promptomatix functions
from promptomatix.main import process_input, generate_feedback, optimize_with_feedback

# Load environment variables
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

print("✅ Advanced features setup complete")

## 1. Custom Configurations and Parameters

Let's explore advanced configuration options for fine-tuning optimization.

In [ ]:
# Advanced configuration with custom parameters
advanced_configs = {
    "conservative": {
        "description": "Conservative approach - low cost, basic optimization",
        "config": {
            "raw_input": "Classify text sentiment",
            "model_name": "gpt-3.5-turbo",
            "model_api_key": api_key,
            "model_provider": "openai",
            "backend": "simple_meta_prompt",
            "synthetic_data_size": 2,
            "task_type": "classification",
            "temperature": 0.1,
            "max_tokens": 1000,
            "train_ratio": 0.7
        }
    },
    "balanced": {
        "description": "Balanced approach - moderate cost and quality",
        "config": {
            "raw_input": "Classify text sentiment",
            "model_name": "gpt-3.5-turbo",
            "model_api_key": api_key,
            "model_provider": "openai",
            "backend": "simple_meta_prompt",
            "synthetic_data_size": 5,
            "task_type": "classification",
            "temperature": 0.3,
            "max_tokens": 1500,
            "train_ratio": 0.8
        }
    },
    "aggressive": {
        "description": "Aggressive approach - high quality, higher cost",
        "config": {
            "raw_input": "Classify text sentiment",
            "model_name": "gpt-4",
            "model_api_key": api_key,
            "model_provider": "openai",
            "backend": "dspy",
            "synthetic_data_size": 10,
            "task_type": "classification",
            "temperature": 0.5,
            "max_tokens": 2000,
            "train_ratio": 0.9
        }
    }
}

config_results = {}

for approach, details in advanced_configs.items():
    print(f"\n🔄 Testing {approach} approach...")
    print(f"Description: {details['description']}")
    
    try:
        result = process_input(**details['config'])
        config_results[approach] = result
        
        print(f"✅ {approach} completed")
        print(f"📝 Prompt length: {len(result['result'])} characters")
        print(f"💰 Cost: ${result['metrics']['cost']:.4f}")
        print(f"⏱️  Time: {result['metrics']['time_taken']:.2f}s")
        print(f"📊 Synthetic data: {len(result['synthetic_data'])} examples")
        
    except Exception as e:
        print(f"❌ {approach} failed: {str(e)}")
        config_results[approach] = None

## 2. Batch Processing and Automation

Let's implement batch processing for multiple prompts.

In [ ]:
# Load sample prompts from the data file
def load_sample_prompts():
    """Load sample prompts for batch processing."""
    prompts = []
    try:
        with open('../data/sample_prompts.txt', 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    prompts.append(line)
    except FileNotFoundError:
        # Fallback prompts if file not found
        prompts = [
            "Classify customer feedback as positive, negative, or neutral",
            "Summarize a long article in 3 sentences",
            "Extract key information from customer reviews",
            "Generate creative marketing copy for a product",
            "Answer questions about a given text accurately"
        ]
    return prompts[:5]  # Limit to 5 for demonstration

sample_prompts = load_sample_prompts()
print(f"📝 Loaded {len(sample_prompts)} sample prompts for batch processing")

# Batch processing function
def process_batch(prompts: List[str], max_workers: int = 2) -> List[Dict]:
    """Process multiple prompts in parallel."""
    results = []
    
    def process_single(prompt: str) -> Dict:
        config = {
            "raw_input": prompt,
            "model_name": "gpt-3.5-turbo",
            "model_api_key": api_key,
            "model_provider": "openai",
            "backend": "simple_meta_prompt",
            "synthetic_data_size": 3,
            "task_type": "classification"
        }
        
        try:
            result = process_input(**config)
            return {"success": True, "result": result, "prompt": prompt}
        except Exception as e:
            return {"success": False, "error": str(e), "prompt": prompt}
    
    print(f"🔄 Processing {len(prompts)} prompts with {max_workers} workers...")
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_single, prompt) for prompt in prompts]
        
        for i, future in enumerate(futures, 1):
            result = future.result()
            results.append(result)
            
            if result['success']:
                print(f"✅ {i}/{len(prompts)}: {result['prompt'][:50]}...")
            else:
                print(f"❌ {i}/{len(prompts)}: {result['error']}")
    
    return results

# Run batch processing
batch_results = process_batch(sample_prompts, max_workers=2)

# Analyze batch results
successful_results = [r for r in batch_results if r['success']]
failed_results = [r for r in batch_results if not r['success']]

print(f"\n📊 Batch Processing Summary:")
print(f"Total prompts: {len(batch_results)}")
print(f"Successful: {len(successful_results)}")
print(f"Failed: {len(failed_results)}")
print(f"Success rate: {len(successful_results)/len(batch_results)*100:.1f}%")

if successful_results:
    total_cost = sum(r['result']['metrics']['cost'] for r in successful_results)
    total_time = sum(r['result']['metrics']['time_taken'] for r in successful_results)
    print(f"Total cost: ${total_cost:.4f}")
    print(f"Total time: {total_time:.2f} seconds")

## 3. Integration Examples

Let's explore how to integrate Promptomatix with other systems.

In [ ]:
# Example: Integration with a simple API wrapper
class PromptomatixAPI:
    """Simple API wrapper for Promptomatix integration."""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.session_history = {}
    
    def optimize_prompt(self, raw_input: str, task_type: str = "classification", **kwargs) -> Dict:
        """Optimize a prompt with error handling and logging."""
        try:
            config = {
                "raw_input": raw_input,
                "model_name": kwargs.get("model_name", "gpt-3.5-turbo"),
                "model_api_key": self.api_key,
                "model_provider": "openai",
                "backend": kwargs.get("backend", "simple_meta_prompt"),
                "synthetic_data_size": kwargs.get("synthetic_data_size", 3),
                "task_type": task_type
            }
            
            result = process_input(**config)
            
            # Store in session history
            self.session_history[result['session_id']] = {
                'raw_input': raw_input,
                'optimized_prompt': result['result'],
                'task_type': task_type,
                'timestamp': time.time(),
                'metrics': result['metrics']
            }
            
            return {
                'success': True,
                'session_id': result['session_id'],
                'optimized_prompt': result['result'],
                'metrics': result['metrics']
            }
            
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'raw_input': raw_input
            }
    
    def get_session_history(self) -> Dict:
        """Get session history."""
        return self.session_history
    
    def get_session_stats(self) -> Dict:
        """Get session statistics."""
        if not self.session_history:
            return {'total_sessions': 0, 'total_cost': 0, 'total_time': 0}
        
        total_cost = sum(session['metrics']['cost'] for session in self.session_history.values())
        total_time = sum(session['metrics']['time_taken'] for session in self.session_history.values())
        
        return {
            'total_sessions': len(self.session_history),
            'total_cost': total_cost,
            'total_time': total_time,
            'average_cost_per_session': total_cost / len(self.session_history)
        }

# Test the API wrapper
print("🔄 Testing API wrapper integration...")

api_wrapper = PromptomatixAPI(api_key)

# Test multiple optimizations
test_inputs = [
    "Analyze customer sentiment",
    "Summarize technical documentation",
    "Extract key metrics from reports"
]

for i, test_input in enumerate(test_inputs, 1):
    print(f"\n📝 Test {i}: {test_input}")
    result = api_wrapper.optimize_prompt(test_input, task_type="classification")
    
    if result['success']:
        print(f"✅ Success - Session: {result['session_id']}")
        print(f"💰 Cost: ${result['metrics']['cost']:.4f}")
    else:
        print(f"❌ Failed: {result['error']}")

# Get session statistics
stats = api_wrapper.get_session_stats()
print(f"\n📊 API Wrapper Statistics:")
print(f"Total sessions: {stats['total_sessions']}")
print(f"Total cost: ${stats['total_cost']:.4f}")
print(f"Total time: {stats['total_time']:.2f} seconds")
print(f"Average cost per session: ${stats['average_cost_per_session']:.4f}")

## 4. Advanced Optimization Strategies

Let's explore advanced strategies for prompt optimization.

In [ ]:
# Multi-stage optimization strategy
def multi_stage_optimization(raw_input: str, stages: int = 3) -> List[Dict]:
    """Perform multi-stage optimization with progressive refinement."""
    results = []
    current_input = raw_input
    
    for stage in range(stages):
        print(f"\n🔄 Stage {stage + 1}/{stages}: Optimizing...")
        
        # Adjust parameters based on stage
        synthetic_data_size = 2 + stage * 2  # Increase data size progressively
        temperature = 0.1 + stage * 0.2  # Increase creativity progressively
        
        config = {
            "raw_input": current_input,
            "model_name": "gpt-3.5-turbo",
            "model_api_key": api_key,
            "model_provider": "openai",
            "backend": "simple_meta_prompt",
            "synthetic_data_size": synthetic_data_size,
            "task_type": "classification",
            "temperature": temperature
        }
        
        try:
            result = process_input(**config)
            results.append({
                'stage': stage + 1,
                'input': current_input,
                'output': result['result'],
                'metrics': result['metrics'],
                'synthetic_data_count': len(result['synthetic_data'])
            })
            
            # Use output as input for next stage
            current_input = result['result']
            
            print(f"✅ Stage {stage + 1} completed")
            print(f"📝 Output length: {len(result['result'])} characters")
            print(f"💰 Cost: ${result['metrics']['cost']:.4f}")
            
        except Exception as e:
            print(f"❌ Stage {stage + 1} failed: {str(e)}")
            break
    
    return results

# Test multi-stage optimization
print("🚀 Testing Multi-Stage Optimization Strategy")
print("=" * 50)

multi_stage_results = multi_stage_optimization(
    "Classify customer feedback with detailed reasoning", 
    stages=3
)

# Analyze multi-stage results
if multi_stage_results:
    print(f"\n📊 Multi-Stage Optimization Summary:")
    print(f"Total stages completed: {len(multi_stage_results)}")
    
    total_cost = sum(r['metrics']['cost'] for r in multi_stage_results)
    total_time = sum(r['metrics']['time_taken'] for r in multi_stage_results)
    
    print(f"Total cost: ${total_cost:.4f}")
    print(f"Total time: {total_time:.2f} seconds")
    
    print(f"\n📈 Progressive Improvement:")
    for i, result in enumerate(multi_stage_results):
        print(f"Stage {result['stage']}: {len(result['output'])} chars, ${result['metrics']['cost']:.4f}")

## 5. Performance Monitoring and Debugging

Let's implement performance monitoring and debugging tools.

In [ ]:
# Performance monitoring class
class PerformanceMonitor:
    """Monitor and track performance metrics."""
    
    def __init__(self):
        self.operations = []
        self.errors = []
    
    def track_operation(self, operation_type: str, start_time: float, end_time: float, 
                       cost: float, success: bool, metadata: Dict = None):
        """Track an operation."""
        operation = {
            'type': operation_type,
            'start_time': start_time,
            'end_time': end_time,
            'duration': end_time - start_time,
            'cost': cost,
            'success': success,
            'metadata': metadata or {}
        }
        
        self.operations.append(operation)
        
        if not success:
            self.errors.append(operation)
    
    def get_performance_summary(self) -> Dict:
        """Get performance summary."""
        if not self.operations:
            return {'total_operations': 0, 'success_rate': 0, 'total_cost': 0}
        
        successful_ops = [op for op in self.operations if op['success']]
        total_cost = sum(op['cost'] for op in self.operations)
        total_time = sum(op['duration'] for op in self.operations)
        
        return {
            'total_operations': len(self.operations),
            'successful_operations': len(successful_ops),
            'failed_operations': len(self.errors),
            'success_rate': len(successful_ops) / len(self.operations),
            'total_cost': total_cost,
            'total_time': total_time,
            'average_cost_per_operation': total_cost / len(self.operations),
            'average_time_per_operation': total_time / len(self.operations)
        }
    
    def get_error_analysis(self) -> Dict:
        """Analyze errors."""
        if not self.errors:
            return {'total_errors': 0, 'error_types': {}}
        
        error_types = {}
        for error in self.errors:
            error_type = error['metadata'].get('error_type', 'unknown')
            error_types[error_type] = error_types.get(error_type, 0) + 1
        
        return {
            'total_errors': len(self.errors),
            'error_types': error_types
        }

# Test performance monitoring
print("🔍 Testing Performance Monitoring...")

monitor = PerformanceMonitor()

# Simulate some operations
test_operations = [
    {"input": "Classify sentiment", "type": "optimization"},
    {"input": "Summarize text", "type": "optimization"},
    {"input": "Invalid input", "type": "optimization"}  # This will fail
]

for i, op in enumerate(test_operations):
    print(f"\n🔄 Operation {i + 1}: {op['input']}")
    
    start_time = time.time()
    
    try:
        if "Invalid" in op['input']:
            raise ValueError("Invalid input detected")
        
        config = {
            "raw_input": op['input'],
            "model_name": "gpt-3.5-turbo",
            "model_api_key": api_key,
            "model_provider": "openai",
            "backend": "simple_meta_prompt",
            "synthetic_data_size": 2,
            "task_type": "classification"
        }
        
        result = process_input(**config)
        end_time = time.time()
        
        monitor.track_operation(
            operation_type=op['type'],
            start_time=start_time,
            end_time=end_time,
            cost=result['metrics']['cost'],
            success=True,
            metadata={'input': op['input']}
        )
        
        print(f"✅ Success - Cost: ${result['metrics']['cost']:.4f}")
        
    except Exception as e:
        end_time = time.time()
        
        monitor.track_operation(
            operation_type=op['type'],
            start_time=start_time,
            end_time=end_time,
            cost=0,
            success=False,
            metadata={'input': op['input'], 'error_type': type(e).__name__, 'error': str(e)}
        )
        
        print(f"❌ Failed: {str(e)}")

# Display performance summary
summary = monitor.get_performance_summary()
error_analysis = monitor.get_error_analysis()

print(f"\n📊 Performance Summary:")
print(f"Total operations: {summary['total_operations']}")
print(f"Success rate: {summary['success_rate']:.1%}")
print(f"Total cost: ${summary['total_cost']:.4f}")
print(f"Total time: {summary['total_time']:.2f} seconds")
print(f"Average cost per operation: ${summary['average_cost_per_operation']:.4f}")

print(f"\n🔍 Error Analysis:")
print(f"Total errors: {error_analysis['total_errors']}")
for error_type, count in error_analysis['error_types'].items():
    print(f"  {error_type}: {count}")

## 6. Best Practices and Recommendations

Based on our advanced feature exploration, here are best practices.

In [ ]:
print("💡 Advanced Features Best Practices:")
print("\n1. Configuration Strategy:")
print("   - Start with conservative settings for testing")
print("   - Use balanced settings for production")
print("   - Apply aggressive settings only for critical tasks")
print("   - Monitor costs and adjust parameters accordingly")

print("\n2. Batch Processing:")
print("   - Use ThreadPoolExecutor for parallel processing")
print("   - Limit max_workers to avoid API rate limits")
print("   - Implement proper error handling and retry logic")
print("   - Track progress and provide user feedback")

print("\n3. Integration Patterns:")
print("   - Create wrapper classes for consistent API")
print("   - Implement session management and history tracking")
print("   - Add comprehensive error handling")
print("   - Provide statistics and monitoring capabilities")

print("\n4. Advanced Optimization:")
print("   - Use multi-stage optimization for complex tasks")
print("   - Progressively increase synthetic data size")
print("   - Adjust temperature for creativity vs consistency")
print("   - Monitor quality improvements vs cost increases")

print("\n5. Performance Monitoring:")
print("   - Track all operations with timing and cost")
print("   - Monitor success rates and error patterns")
print("   - Set up alerts for cost thresholds")
print("   - Analyze performance trends over time")

print("\n6. Production Considerations:")
print("   - Implement rate limiting and backoff strategies")
print("   - Add comprehensive logging and debugging")
print("   - Set up monitoring and alerting systems")
print("   - Plan for scalability and cost management")

## Summary

In this notebook, we explored advanced features of Promptomatix:

✅ **Custom Configurations**: Explored different optimization approaches
✅ **Batch Processing**: Implemented parallel processing for multiple prompts
✅ **Integration Examples**: Created API wrappers and session management
✅ **Advanced Strategies**: Multi-stage optimization and progressive refinement
✅ **Performance Monitoring**: Comprehensive tracking and debugging tools
✅ **Best Practices**: Production-ready recommendations

### Key Advanced Features:

- **Multi-Stage Optimization**: Progressive refinement for complex tasks
- **Batch Processing**: Efficient handling of multiple prompts
- **API Integration**: Wrapper classes for consistent interfaces
- **Performance Monitoring**: Comprehensive tracking and analysis
- **Error Handling**: Robust error management and debugging

### Production Readiness:

The advanced features demonstrated here provide the foundation for:
- Scalable prompt optimization systems
- Production-ready integrations
- Comprehensive monitoring and debugging
- Cost-effective batch processing

### Next Steps:

- Implement these patterns in your own applications
- Set up monitoring and alerting systems
- Optimize for your specific use cases
- Consider custom metrics and evaluation frameworks

---

**Congratulations!** You've now explored all the advanced features of Promptomatix. You're ready to build production-ready prompt optimization systems!